In [ ]:
import functools # used to maintain introspection on decorators
import requests
import urllib
import time
import uuid # used to create a unique 'computer' id for each recipe
import json # used to store the scraped details
import os

from uuid import UUID # used to create a unique id for each recipe
from json import JSONEncoder # used to convert the UUID into a writable format
from urllib.request import Request, urlopen

#from decorators import noSuchElementException

from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC


#from selenium.webdriver.chrome.options import Options
#from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome()

class data:

    articles = [] # Used to make a list of recipes
    button = None # Used to interact with various button elements
    container = None # Used to store various container elements
    currentURL = "" # Used to store various urls 
    pages = [] # Used to append a list with pages links
    recipeLinks = [] # Used to store recipe links
    recipeName = "" # Stores the recipe name
    searchbar = None # Used to interact with search bar
    source = "" # Used to get page source code
    tag = None # Used to store various tag elements
    title = "" # Used to get the title
    totalPages = [] # Stores a list of pages

    # File Management
    count = 0 # Used in the creation of image filenames
    dataDirectory = "" # Used to create folder
    imageDirectory = "" # Used to create folder 
    recipeDirectory = "" # Used to create modified folder names

    # Scraped Information
    recipeDetails = {} # Used to store all the scraped recipe details

    allergens = "" # Used to store scraped allergens
    alternatives = "" #Used to store scraped alternatives
    description = "" # Used to store the scraped description of the recipe
    freeFrom = "" # Used to store the scraped free from information
    imageLinks = [] # Used to scrape all of a recipes image links
    ingredients = "" # Used to store the scraped ingredients
    instructions = "" # Used to store scraped instructions
    mainPhoto = None # Used to store main photo link
    name = "" # Used to store scraped recipe name
    notes = "" # Used to store scraped recipe notes
    recipeTags = "" # Used to store scraped recipe tags
    storage = "" # Used to store scraped storage instructions
    timeCook = "" # Used to store scraped cook time
    timePrep = "" # Used to store scraped recipe  prep time 
    timeTotal = "" # Used to store scraped total time it takes to make the recipe

class scraper:
    def intitialize(self, url, searchTerm, delay):
        global_ids = scraper.getUniqueID(scraper, 'https://www.pickuplimes.com/recipe/spicy-garlic-wok-noodles-213')
    
        self.getURL(url) # Have to start somewhere
        self.run(self)
        self.closeSession() # Have to end somewhere
    
    # DECORATORS

    def exceptionHandling(func):
        @functools.wraps(func) # maintains introspection
        def wrapper(*args, **kwargs):
            try:
                func(*args, **kwargs)
            except NoSuchElementException:
                print(f"{func.__name__} Exception: Element Not Found")
            except TimeoutException:
                print(f"{func.__name__} Exception: Timeout")
            return func(*args, **kwargs)
        return wrapper

    def functionTimer(func):
        """Prints the functions runtime"""
        @functools.wraps(func) # maintains introspection
        def wrapper(*args, **kwargs):
            start = time.perf_counter()
            func(*args, **kwargs)
            end = time.perf_counter()
            runtime = end - start

            print (f"Finished {func.__name__!r} in {runtime: .4f} secs")
            return func(*args, **kwargs)
        return wrapper

    # Best applied to small convenience functions that you don’t call directly yourself
    def debug(func):
        """Print the function signature and return value"""
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            args_repr = [repr(a) for a in args]                     # Creates a list of positional arguments, repr() returns a string for each argument               
            kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()] # Creates a list of keyword arguments, f-string formats each argument as key=value, !r specifier means that repr() is used to represent the value
            signature = ", ".join(args_repr + kwargs_repr)          # Join both lists together to make a signature
            print(f"Calling {func.__name__}({signature})")
            func(*args, **kwargs)
            print(f"{func.__name__!r} returned {func!r}")           
            return func
        return wrapper

    # Best applied to functions you only need to call less often
    def slowDown(func):
        '''Sleep before calling the function'''
        @functools.wraps(func) # maintains introspection
        def wrapper(*args, **kwargs):
            time.sleep(1)
            return func(*args, **kwargs)
        return wrapper

    def scrapeHandling(element):
        def wrapperOuter(func):
            @functools.wraps(func) # maintains introspection
            def wrapperInner(*args, **kwargs):
                try:
                    func(*args, **kwargs)
                except NoSuchElementException:
                    print(f"{func.__name__} Exception:", element, "Not Found")
                    element = "N/A"
                except TimeoutException:
                    print(f"{func.__name__} Exception: Timeout")
                    element = "N/A"
                return func
            return wrapperInner
        return wrapperOuter

    def folderAlreadyExists(folderName):
        def wrapperOuter(func):
            @functools.wraps(func) # maintains introspection
            def wrapperInner(*args, **kwargs):
                try:
                    func(*args, **kwargs)
                except:
                    print(f"{func.__name__} Error")
                    print(folderName, "Folder Already Exists")
                return func
            return wrapperInner
        return wrapperOuter
    
    # Counts the number of time the function has been called
    def callCount(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            wrapper.calls += 1
            print(f"Call {wrapper.calls} of {func.__name__!r}")
            return func(*args, **kwargs)
        wrapper.calls = 0
        return wrapper

    def run(self):
        #self.acceptCookies()
        data.currentURL = self.findRecipeList(self)
        self.getAllRecipePages(self, data.currentURL)
        self.getRecipes(self, data.currentURL)
        self.cycleRecipeLinks(self)
        self.closeSession()   

    def cycleRecipeLinks(self):
        for i in data.recipeLinks:
            data.currentURL = i
            self.makeImage(self, data.currentURL)

    #@callCount
    def getURL(url):
        '''Navigates to a website using a url passed as a perameter.'''
        driver.get(url) 

    def getTitle():
        '''Fetches the title and prints it to screen.'''
        data.title = driver.title

    def closeSession():
        '''Closes the driver'''
        driver.quit()

    def getSourceCode():
        '''Fetches source code for the page.'''
        data.source = driver.page_source

    @exceptionHandling
    def search(self, searchTerm):
        '''Finds search bar, types in the search term which it takes as a perameter and clicks to navigate to the next page.'''
        button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, 'nav-searchbar-btn')))
        button.click()

        self.findSearchbar(self, searchTerm)

    def searchbarTextAndClick(searchTerm):
        try:
            data.searchbar.send_keys(searchTerm)
            data.searchbar.send_keys(Keys.RETURN) # Return = Enter
        except:
            print("Exception: No search term input")
    
    @exceptionHandling
    def findSearchbar(self, searchTerm):

        data.searchbar = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.NAME, "sb")))
        self.searchbarTextAndClick(searchTerm)

    @exceptionHandling
    def home():
        '''Finds the title and clicks it.'''
        title = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, 'nav-image')))
        title.click()

    @exceptionHandling
    def findRecipeList(self):
        '''Finds the recipe tab and clicks it.'''
        data.button = WebDriverWait(driver,5).until(EC.presence_of_element_located((By.LINK_TEXT, 'Recipes')))
        data.button.click()

    @exceptionHandling
    def acceptCookies():
        '''Finds the accept cookies button and clicks it.'''
        data.button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div[2]/div[2]')))
        data.button.click()
    
    def getRecipes(self, url):
        '''Finds the recipe container and puts all the recipes in a list.'''

        self.getRecipeContainer()
        self.makeRecipeList()

    @exceptionHandling
    def getRecipeContainer():
        data.container = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//*[@id='index-item-container']/div/div[2]/ul"))) 

    def makeRecipeList():
        data.articles = data.container.find_elements(By.TAG_NAME, 'li')

        for i in data.articles:
            data.tag = i.find_element(By.TAG_NAME, 'a')
            data.recipeLinks.append(data.tag.get_attribute('href'))

    def getPageURL():
        '''Returns the current page url.'''
        data.currentURL =  driver.current_url

    def getAllRecipePages(self, url):
        '''Navigates to each recipe page by modifying the current url and stores them in a list.'''

        self.getTotalPages()
        self.getSearchList()

    @exceptionHandling
    def getTotalPages():
         #totalPages = driver.find_element(By.CLASS_NAME, 'page-text') #actual
        data.totalPages = [1, 2, 3] #temp to shorten runtime

    def getSearchList():
        for i in data.totalPages:
            data.currentURL = driver.current_url
            url_change = "?page=" + str(i)
            next_page = data.currentURL + url_change
            data.pages.append(next_page)

    def getUniqueID(self, url):
        '''Creates a uuid for each recipe taking a url as a perameter'''
        
        page_ID = url
        just_ID = page_ID.replace(str("https://www.pickuplimes.com/recipe/"), "")

        ids = (just_ID, uuid.uuid4())

        return ids

    @scrapeHandling(data.name)
    def scrapeName():

        data.name = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="header-info-col"]/div/header/h1'))).text

    @scrapeHandling(data.recipeTags)
    def scrapeTags():

        data.recipeTags = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="header-info-col"]/div/header/a[1]/div/p'))).text

    @scrapeHandling(data.description)
    def scrapeDescription():

        data.description = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="header-info-col"]/div/header/span'))).text

    @scrapeHandling(data.timeTotal)
    def scrapeTotalTime():
        
        data.timeTotal = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="recipe-info-container"]/div[2]'))).text  

    @scrapeHandling(data.timePrep)
    def scrapePrepTime():

        data.timePrep = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="recipe-info-container"]/div[3]'))).text

    @scrapeHandling(data.timeCook)
    def scrapeCookTime():
        
        data.timeCook = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="recipe-info-container"]/div[4]'))).text

    @scrapeHandling(data.allergens)
    def scrapeAllergens():

        data.allergens = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="allergen-info-container"]/div[1]/div'))).text 

    @scrapeHandling(data.alternatives)
    def scrapeAlternatives():
        
        data.alternatives = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="allergen-info-container"]/div[2]/div'))).text

    @scrapeHandling(data.freeFrom)
    def scrapeFreeFrom():
        
        data.freeFrom = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="allergen-info-container"]/div[3]/div'))).text 

    @scrapeHandling(data.ingredients)
    def scrapeIngredients():
        
        data.ingredients = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ingredient-direction-container"]/div/div[2]'))).text

    @scrapeHandling(data.instructions)
    def scrapeInstructions():
        
        data.instructions = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ingredient-direction-container"]/div/div[4]/section/ol'))).text

    @scrapeHandling(data.notes)
    def scrapeNotes():
        
        data.notes = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ingredient-direction-container"]/div/div[4]/section/ul[1]/li'))).text

    @scrapeHandling(data.storage)
    def scrapeStorage():
        
        data.storage = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ingredient-direction-container"]/div/div[4]/section/ul[2]/li'))).text

    @scrapeHandling(data.mainPhoto)
    def scrapeMainPhoto():
        
        data.mainPhoto = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="main-image-container"]/img')))


    @exceptionHandling
    def scrapeImages():
        imageContainer = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="recipe-video"]/div[2]'))) # Find the container
        imageList = imageContainer.find_elements(By.XPATH, 'img') # Find the children

        for i in imageList:
            link = i.get_attribute('src')
            data.imageLinks.append(link)

    @exceptionHandling
    def getRecipeDetails(self, url):
        self.getURL(url)

        self.scrapeName()  
        self.scrapeTags()
        self.scrapeDescription()
        self.scrapeTotalTime()
        self.scrapePrepTime()
        self.scrapeCookTime()
        self.scrapeAllergens()
        self.scrapeAlternatives()
        self.scrapeFreeFrom()
        self.scrapeIngredients()
        self.scrapeInstructions()
        self.scrapeNotes()
        self.scrapeStorage()
        self.scrapeMainPhoto()
        self.scrapeImages()

        self.storeDetails(self, url)
        self.jsonFile(self)

    def storeDetails(self, url):
        data.recipeDetails = {'ID': [], 'Name': [], 'Photo': [],'Tags': [], 'Description': [], 'Total Time': [], 'Prep Time': [], 'Cook Time': [], 'Allergens': [], 'Swaps': [], 'Free From': [], 'Ingredients': [], 'Directions': [], 'Notes': [], 'Storage': [], 'Images': []}
        data.recipeDetails['ID'].append(self.getUniqueID(self, url))
        data.recipeDetails['Name'].append(data.name)
        data.recipeDetails['Photo'].append(data.mainPhoto)
        data.recipeDetails['Tags'].append(data.recipeTags)
        data.recipeDetails['Description'].append(data.description)
        data.recipeDetails['Total Time'].append(data.timeTotal)
        data.recipeDetails['Prep Time'].append(data.timePrep)
        data.recipeDetails['Cook Time'].append(data.timeCook)
        data.recipeDetails['Allergens'].append(data.allergens)
        data.recipeDetails['Swaps'].append(data.alternatives)
        data.recipeDetails['Free From'].append(data.freeFrom)
        data.recipeDetails['Ingredients'].append(data.ingredients)
        data.recipeDetails['Directions'].append(data.instructions)
        data.recipeDetails['Notes'].append(data.notes)
        data.recipeDetails['Storage'].append(data.storage)
        data.recipeDetails['Images'].append(data.imageLinks)

    def jsonFile(self):
        '''Creates a folder called 'raw_data' in the path for the json file to be saved in
        Uses a try except catch as it will throw an error if the folder already exists'''

        self.makeRaw_DataFolder()

        '''Deals with TypeError: Object of type UUID is not JSON serializable by encoding the UUID'''
        JSONEncoder_olddefault = JSONEncoder.default
        def JSONEncoder_newdefault(self, o):
            if isinstance(o, UUID): return str(o)
            return JSONEncoder_olddefault(self, o)
        JSONEncoder.default = JSONEncoder_newdefault

        self.jsonDump()

    @folderAlreadyExists("raw_data")
    def makeRaw_DataFolder():
        
        data.dataDirectory = "raw_data"
        parent_dir = "C:/Users/Millie/Documents/AiCore/AiCore/DataCollectionPipeline"
        path = os.path.join(parent_dir, data.dataDirectory)
        os.mkdir(path)
        print("Directory '% s' created" % data.dataDirectory)
    
    def jsonDump():
        '''Stores data by writing the 'recipe_details' dictionary to a JSON file called 'data.json' in the folder just created
        The dicrionary is converted to a string using str() to deal with 'TypeError: Object of type WebElement is not JSON serializable'''
        with open(os.path.join('raw_data', 'data.json'), 'w') as json_file:
            json.dump(str(data.recipeDetails), json_file)

    def downloadImage(self, url, recipeName):
        '''Creates a folder called 'images' and another with the recipe name in the path for the image files to be saved in
        Uses a try except catch as it will throw an error if the folders already exists
        Adds User-Agent Headers to bypass 403 error
        Downloads the images into the folder of that recipe name'''

        self.makeImagesFolder()
        self.makeRecipeFolder()
        
        try:
            # Adds headers to resolve 403 Fobidden Error
            opener=urllib.request.build_opener()
            opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
            urllib.request.install_opener(opener)

            downloadDirectory = "images/" + data.recipeDirectory + "/"
            fileType = '.jpg'
            fileName = downloadDirectory + recipeName + fileType
            image = urllib.request.urlretrieve(url, fileName)
        except:
            print("Error Downloading Images")           

    @folderAlreadyExists("Images")
    def makeImagesFolder():
        
        data.imageDirectory = "images"
        parent_dir = "C:/Users/Millie/Documents/AiCore/AiCore/DataCollectionPipeline"
        path = os.path.join(parent_dir, data.imageDirectory)
        os.mkdir(path)
        print("Directory '% s' created" % data.imageDirectory)

    @folderAlreadyExists("Recipe")
    def makeRecipeFolder():

        data.recipeDirectory = data.recipeName.replace(".jpg", "").replace("0", "").replace("1", "").replace("2", "").replace("3", "").replace("4", "").replace("5", "").replace("6", "").replace("7", "").replace("8", "").replace("9", "")
        parent_dir = "C:/Users/Millie/Documents/AiCore/AiCore/DataCollectionPipeline/images"
        path = os.path.join(parent_dir, data.recipeDirectory)
        os.mkdir(path)
        print("Directory '% s' created" % data.recipeDirectory)

    def makeImage(self, url):
        '''Retrieves the ID of each image using 'getRecipeDetails()
        Removes all unecissary elements from the ID string to create a file name
        Pass the file name to 'downloadImages() to create a file'''
        
        self.getRecipeDetails(self, url)
        
        for i in data.recipeDetails['Images']:

            for j in i:
                IDtoName = data.recipeDetails['ID'][0]
                IDtoName = IDtoName[0].replace("-", " ")
                IDtoName = IDtoName.title()

                for i in IDtoName:
                    if i.isdigit():
                        IDtoName = IDtoName.replace(i, "")

        s = str(data.count)
        data.recipeName = IDtoName + s + ".jpg"
        self.downloadImage(self, j, data.recipeName)
        data.count = data.count + 1                

scraper.intitialize(scraper, 'https://www.pickuplimes.com', 'lemons', 5)

# IDEALS ---------------------------------------------------------------------------------------------------------------
# link getTitle() and home()
# link getPageURL() and getUniqueID()
# get search results
# get search results on more than the first page
# make a separate main image
# counts how many recipes there are on the site
# a for loop that automaticaly changes the xpath to get all the images for the recipe
# bypass login
# replace all XPaths with written XPaths
# a method that makes a list containing all buttons and cycles through them looking for a link_text given as an argument
# seperate website specific methods from general functioning in a class
# make sure all code works with any website so is reusable and genralisable
# use the unique id (page url) to stop scraping recipes it has already scraped 
# have a wait for element function that takes a xpath and a perameter
# time each function to optimise run speed


In [100]:
import os # for testing directories
import requests # for testing website response
import unittest
import uuid # for testing the recipe unique identifyer
from scraper import scraper

class scraperTestCase(unittest.TestCase):

    recipe = 'harissa-spiced-beans-898-0.jpg'
    directory = 'C:/Users/Millie/Documents/AiCore/AiCore/DataCollectionPipeline'

    @classmethod
    def setUpClass(cls): # Runs at the begining of the file
        pass

    @classmethod
    def tearDownClass(cls): # Runs at the end of the file
        pass

    def setUp(self): # Runs before every test
        self.bot1 = scraper() 
        self.friendlyID = 'harissa-spiced-beans-898'
        self.systemID = uuid.uuid4()
        self.dictionary = {"ID": []}
        self.number = str(3)
        self.folderName = 'images'
        #self.handle = open("data.json", "r") # Dont forget to test the json file

    def tearDown(self): # Runs at the end of every test
        del self.bot1
    
    def test_dataClass(self):
        pass
    
    def test_decoratorClass(self):
        pass

    def test_exceptionHandling(self):
        pass

    def test_run(self):
        pass

    def test_cycleRecipeLinks(self):
        pass

    def test_getURL(self):
        print("test_getURL")
        response = requests.get('https://www.pickuplimes.com/')

        if response.ok:
            print("Website Found")
            return response.text
        else:
            print("!Website Not Found!")
            return 'Bad Response'

    def test_getTitle(self):
        pass

    def test_closeSession(self):
        pass

    def test_getSourceCode(self):
        pass

    def test_search(self):
        pass

    def test_searchbarTextAndClick(self):
        pass

    def test_findSearchbar(self):
        pass

    def test_home(self):
        pass

    def test_findRecipeList(self):
        pass

    def test_acceptCookies(self):
        pass

    def test_getRecipes(self):
        pass

    def test_getRecipeContainer(self):
        pass

    def test_makeRecipeList(self):
        pass

    def test_getPageURL(self):
        pass

    def test_getAllRecipePages(self):
        pass

    def test_getTotalPages(self):
        pass

    def test_getSearchList(self):
        pass

    def test_getUniqueID(self):
        pass

    def test_scrapeName(self):
        pass

    def test_scrapeTags(self):
        pass

    def test_scrapeDescription(self):
        pass

    def test_scrapeTotalTime(self):
        pass

    def test_scrapePrepTime(self):
        pass

    def test_scrapeCookTime(self):
        pass

    def test_scrapeAllergens(self):
        pass

    def test_scrapeAlternatives(self):
        pass

    def test_scrapeFreeFrom(self):
        pass

    def test_scrapeIngredients(self):
        pass

    def test_scrapeInstructions(self):
        pass

    def test_scrapeInstructions(self):
        pass

    def test_scrapeNotes(self):
        pass

    def test_scrapeStorage(self):
        pass

    def test_scrapeMainPhoto(self):
        pass

    def test_scrapeImages(self):
        pass

    def test_getRecipeDetails(self):
        pass

    def test_storeDetails(self):
        pass

    def test_jsonFile(self):
        pass

    def test_makeRaw_DataFolder(self):
        pass

    def jsonDump(self):
        pass

    def test_downloadImage(self):
        pass

    # UNFINISHED
    def test_makeImagesFolder(self):
        print('test_makeImagesFolder')       

        # Check directory is correct
        imagePath = os.path.join(self.directory, self.folderName)
        self.assertEqual(imagePath, 'C:/Users/Millie/Documents/AiCore/AiCore/DataCollectionPipeline\images') 

        # Check folder was created

    def test_makeRecipeFolder(self):
        print('test_makeRecipeFolder')

        recipePath = os.path.join(self.directory, self.folderName)

        self.assertEqual(self.recipe.replace(".jpg", "").replace("0", ""), 'harissa-spiced-beans-898-')
        self.assertEqual(recipePath, 'C:/Users/Millie/Documents/AiCore/AiCore/DataCollectionPipeline\images')
        self.assertEqual(os.path.join(recipePath, self.recipe), 'C:/Users/Millie/Documents/AiCore/AiCore/DataCollectionPipeline\images\harissa-spiced-beans-898-0.jpg')

    def test_makeImage(self):
        print("test_makeImage")

        self.dictionary['ID'].append(self.friendlyID)
        self.dictionary['ID'].append(self.systemID)

        # Check the dictionary contains the correct IDs
        self.assertEqual(self.dictionary['ID'][0], self.friendlyID)
        self.assertAlmostEqual(self.dictionary['ID'][1], self.systemID)

        # Check it removes all unnecessary bits from the string
        editedID = (self.dictionary['ID'][0])
        editedID = editedID.title()
        editedID = editedID.replace("-", " ")
        for i in editedID:
            if i.isdigit():
                editedID = editedID.replace(i , "")

        # Check the finished string is correct
        name = editedID + self.number + ".jpg"
        self.assertEqual(name, editedID + self.number + ".jpg")     

unittest.main(argv=['first-arg-is-ignored'], exit=False)

#if __name__ == "__main__":
#    test_scraper() 
#    print("Passed")

..............................................

test_getURL
Website Found
test_makeImage
test_makeImagesFolder
test_makeRecipeFolder



----------------------------------------------------------------------
Ran 46 tests in 0.177s

OK
